## LFP

### This notebook shows an example where a set of electrodes are selected from a dataset and then LFP is extracted from those electrodes and then written to a new NWB file


In [ ]:
import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

## We also import a bunch of tables so that we can call them easily
from spyglass.common import (RawPosition, HeadDir, Speed, LinPos, StateScriptFile, VideoFile,
                                  DataAcquisitionDevice, CameraDevice, Probe,
                                  DIOEvents,
                                  ElectrodeGroup, Electrode, Raw, SampleCount,
                                  LFPSelection, LFP, LFPBandSelection, LFPBand,
                                  SortGroup, SpikeSorting, SpikeSorter, SpikeSorterParameters, SpikeSortingWaveformParameters, SpikeSortingParameters, SpikeSortingMetrics, CuratedSpikeSorting,\
                                  FirFilter,
                                  IntervalList, SortInterval,
                                  Lab, LabMember, LabTeam, Institution,
                                  BrainRegion,
                                  SensorData,
                                  Session, ExperimenterList,
                                  Subject,
                                  Task, TaskEpoch,
                                  Nwbfile, AnalysisNwbfile, NwbfileKachery, AnalysisNwbfileKachery,
                                  interval_list_contains,
                                  interval_list_contains_ind,
                                  interval_list_excludes,
                                  interval_list_excludes_ind,
                                  interval_list_intersect,
                                  get_electrode_indices)

import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [ ]:
nwb_file_names = Nwbfile().fetch('nwb_file_name')
# take the first one for this demonstration
nwb_file_name = nwb_file_names[0]
print(nwb_file_name)


#### Create the standard LFP Filters. This only needs to be done once.

In [ ]:
FirFilter().create_standard_filters()

#### Now we Select every 16th electrode for LFP or, below, a specific set of electrodes. Choose one
Note that this will delete the current selection, and all downstream LFP and LFPBand information (if it exists), but only for the current dataset. This is fine to do if you want to generate or regenerate the LFP

In [ ]:
electrode_ids = (Electrode & {'nwb_file_name' : nwb_file_name}).fetch('electrode_id')
lfp_electrode_ids = electrode_ids[range(0, len(electrode_ids), 128)]
LFPSelection().set_lfp_electrodes(nwb_file_name, lfp_electrode_ids.tolist())


In [ ]:
LFPSelection().LFPElectrode() & {'nwb_file_name' : nwb_file_name}

### Or select one electrode for LFP


In [ ]:
LFPSelection().set_lfp_electrodes(nwb_file_name, [0, 1])

In [ ]:
LFPSelection().LFPElectrode() & {'nwb_file_name':nwb_file_name}

### Populate the LFP table. Note that this takes 2 hours or so on a laptop if you use all electrodes

In [ ]:
LFP().populate([LFPSelection & {'nwb_file_name':nwb_file_name}])

### Now that we've created the LFP object we can perform a second level of filtering for a band of interest, in this case the theta band
We first need to create the filter

In [ ]:
lfp_sampling_rate = (LFP() & {'nwb_file_name' : nwb_file_name}).fetch1('lfp_sampling_rate')
filter_name = 'Theta 5-11 Hz'
FirFilter().add_filter(filter_name, lfp_sampling_rate, 'bandpass', [4, 5, 11, 12], 'theta filter for 1 Khz data')

In [ ]:
FirFilter()

Next we add an entry for the LFP Band and the electrodes we want to filter

In [ ]:
# assume that we've filtered these electrodes; change this if not
lfp_band_electrode_ids = [1]

# set the interval list name corresponding to the second epoch (a run session)
interval_list_name = '02_r1'

# set the reference to -1 to indicate no reference for all channels
ref_elect = [-1]

# desired sampling rate
lfp_band_sampling_rate = 100

In [ ]:
LFPBandSelection().set_lfp_band_electrodes(nwb_file_name, lfp_band_electrode_ids, filter_name, interval_list_name, ref_elect, lfp_band_sampling_rate)

Check to make sure it worked

In [ ]:
(LFPBandSelection() & {'nwb_file_name' : nwb_file_name})

In [ ]:
LFPBand().populate(LFPBandSelection() & {'nwb_file_name' : nwb_file_name})
LFPBand()

### Now we can plot the original signal, the LFP filtered trace, and the theta filtered trace together.
Much of the code below could be replaced by a function calls that would return the data from each electrical series

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#get the three electrical series objects and the indeces of the electrodes we band pass filtered
orig_eseries = (Raw() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['raw']
orig_elect_indeces = get_electrode_indices(orig_eseries, lfp_band_electrode_ids)

lfp_eseries = (LFP() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['lfp']
lfp_elect_indeces = get_electrode_indices(lfp_eseries, lfp_band_electrode_ids)

lfp_band_eseries = (LFPBand() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['filtered_data']
lfp_band_elect_indeces = get_electrode_indices(lfp_band_eseries, lfp_band_electrode_ids)


In [ ]:
# get a list of times for the first run epoch and then select a 2 second interval 100 seconds from the beginning
run1times = (IntervalList & {'nwb_file_name': nwb_file_name, 'interval_list_name' : '02_r1'}).fetch1('valid_times')
plottimes = [run1times[0][0] + 101, run1times[0][0] + 102]

In [ ]:
# get the time indeces for each dataset
orig_time_ind = np.argwhere(np.logical_and(orig_eseries.timestamps > plottimes[0], orig_eseries.timestamps < plottimes[1]))

lfp_time_ind = np.argwhere(np.logical_and(lfp_eseries.timestamps > plottimes[0], lfp_eseries.timestamps < plottimes[1]))
lfp_band_time_ind = np.argwhere(np.logical_and(lfp_band_eseries.timestamps > plottimes[0], lfp_band_eseries.timestamps < plottimes[1]))

In [ ]:
plt.plot(orig_eseries.timestamps[orig_time_ind], orig_eseries.data[orig_time_ind,orig_elect_indeces[0]], 'k-')
plt.plot(lfp_eseries.timestamps[lfp_time_ind], lfp_eseries.data[lfp_time_ind,lfp_elect_indeces[0]], 'b-')
plt.plot(lfp_band_eseries.timestamps[lfp_band_time_ind], lfp_band_eseries.data[lfp_band_time_ind,lfp_band_elect_indeces[0]], 'r-')
plt.xlabel('Time (sec)')
plt.ylabel('Amplitude (AD units)')

plt.show()